In [1]:
from connection import ConnectionTask, pg_select
import time
import tablib
import requests
import json
import random
from pprint import pprint
c=ConnectionTask()

In [ ]:
# 51返呗Id：664的滑动，已领卷但是未使用的用户注册手机号。
app_code='fdbank'
activity_id='1248'
expire_time = time.mktime((2018,3,26,0,0,0,0,0,0))
rdb=c.prdb_activity(0)
user_id_set=set()
count = 0
for key in rdb.scan_iter('c:{}:*'.format(app_code)):
    if 'use' in key:
        continue
    count +=1

    value,timestamp=rdb.zscan(key)[1][0]
    if value.split('|')[-1]==activity_id:
        if timestamp>expire_time:
            continue

        if 'expired' in key:
            user_id_set.add(key.split(':')[-2])
        else:
            user_id_set.add(key.split(':')[-1])
            
    print(count,end='\r')
            
import tablib
user=tablib.Dataset(headers=['用户ID','用户手机号'])
for user_id in user_id_set:
    try:
        phone=c.pmdb.user_customer.find_one({'customerUser.{}'.format(app_code):user_id},{'_id':0,'phone':1})['phone']
    except:
        phone=''
    user.append([user_id,phone])
    print(user_id,end='\r')
with open(f'已领卷未使用的用户手机号{app_code}_{activity_id}.xlsx', 'wb') as f:
    f.write(user.export('xlsx'))
print('ok')

In [ ]:
len(user_id_set)

In [ ]:
import tablib
user=tablib.Dataset(headers=['用户ID','用户手机号'])
for user_id in user_id_set:
    try:
        phone=c.pmdb.user_customer.find_one({'customerUser.{}'.format(app_code):user_id},{'_id':0,'phone':1})['phone']
    except:
        phone=''
    user.append([user_id,phone])
    print(user_id,end='\r')
with open(f'已领卷未使用的用户手机号{app_code}_{activity_id}.xlsx', 'wb') as f:
    f.write(user.export('xlsx'))
print('ok')

In [ ]:
with open('51ala_664.xls', 'wb') as f:
    f.write(user.export('xls'))

In [ ]:
len(user.dict)

In [4]:
# 运营生成卷码\
# 活动id：779  优惠券码760份，id 778，优惠券码3010份
import requests
import tablib
COUPON_URL='http://sessionapi.localdomain:8011/get_coupon_codes/'
# 1251，bsb ，11700个券码
app_code='bsb'
activity_num=[
    {'activity_id':1307,'num':4200},
]

data_book=tablib.Databook()
for activity in activity_num:
    url=COUPON_URL+'{}/{}/{}'.format(app_code,activity['activity_id'],activity['num'])
    resp_data=requests.get(url).json()
#     {
#         'data': {
#             'codes': ['f3e979c46f88c0b5']
#         }, 
#         'message': 'success', 
#         'code': 0
#     }
    print(resp_data['message'])
    dataset=tablib.Dataset(header=['appCode','activityId','couponCode'])
    for item in resp_data['data']['codes']:
        dataset.append([app_code,activity['activity_id'],item])
    dataset.title=str(activity['activity_id'])
    data_book.add_sheet(dataset)
    print(activity,'ok')
with open('coupon-{}-{}.xls'.format(app_code,'-'.join([str(activity['activity_id']) for activity in activity_num])), 'wb') as f:
    f.write(data_book.xls)

success
{'activity_id': 1307, 'num': 4200} ok


{'ets': 1525449599000, 'app_id': 175}


In [26]:
# 运营生成卷码
import requests
import tablib
COUPON_URL='http://promotionapi.localdomain:8011/get_coupon_code_list'
conn = c.ppgdb('oto_saas')

activity_num=[
    {'activityId':1321, 'appCode': 'bsb', 'num':600},
    {'activityId':1323, 'appCode': 'bsb', 'num':400},
]
for an in activity_num:
    for item in pg_select("SELECT ets,app_id FROM activity WHERE id = '{}';".format(an['activityId']),conn):
        item['appId'] = item['app_id']
        an.update(item)

data_book=tablib.Databook()
for activity in activity_num:
    resp_data=requests.post(COUPON_URL,json=activity).json()
#     {
#         'data': {
#             'codes': ['f3e979c46f88c0b5']
#         }, 
#         'message': 'success', 
#         'code': 0
#     }
    print(resp_data['message'])
    dataset=tablib.Dataset(headers=['appCode','activityId','couponCode'])
    for item in resp_data['data']['codes']:
        dataset.append([app_code,activity['activityId'],item])
    dataset.title=str(activity['activityId'])
    data_book.add_sheet(dataset)
    print(activity,'ok')
with open('coupon-{}-{}.xls'.format(app_code,'-'.join([str(activity['activityId']) for activity in activity_num])), 'wb') as f:
    f.write(data_book.xls)

success
{'activityId': 1321, 'appCode': 'bsb', 'num': 600, 'ets': 1527782399000, 'app_id': 175, 'appId': 175} ok
success
{'activityId': 1323, 'appCode': 'bsb', 'num': 400, 'ets': 1527782399000, 'app_id': 175, 'appId': 175} ok


In [ ]:
# 基于mongo.timeline统计ele外卖失败订单原因
import tablib,time

start=time.mktime((2017,11,1,0,0,0,0,0,0))*1000
end=time.mktime((2017,12,12,0,0,0,0,0,0))*1000
counter={}
for item in c.pmdb.order_waimai.find({'createdAt':{'$gte':start,'$lt':end},'status':{'$nin':[1,2,3,4]}},{'id':1,'timeline':1,'_id':0}):
    for time_line in item.get('timeline',[]):
        if '订单失败' in time_line and '支付超时' not in time_line:
            date,*args,reason=time_line.strip(' ').split(' ',3)
            if len(reason)<=2:
                print(time_line,reason)
            counter.setdefault(date,{})
            counter[date].setdefault(reason,0)
            counter[date][reason]+=1
            break

tab_data=tablib.Dataset(headers=['时间','原因','单数'])
for key,value in counter.items():
    for k,v in value.items():
        tab_data.append([key,k,v])

with open('waimai_refund_order_counter_{}-{}.xlsx'.format(
    time.strftime('%Y%m%d',time.localtime(start/1000)),
    time.strftime('%Y%m%d',time.localtime(end/1000))),'wb') as f:
    f.write(tab_data.xlsx)

In [ ]:
# 基于mongo.timeline统计ele外卖失败订单原因
# 把11.1-12.10号的订单中原因是
# 【商家超时未接单】、【暂时无法配送】
# 这两个原因的订单明细导出一份

import tablib,time

start=time.mktime((2017,11,1,0,0,0,0,0,0))*1000
end=time.mktime((2017,12,10,24,0,0,0,0,0))*1000
counter=[]
for item in c.pmdb.order_waimai.find({'createdAt':{'$gte':start,'$lt':end},'status':{'$nin':[1,2,3,4]}},{'_id':0}):
    for time_line in item.get('timeline',[]):
        if '订单失败' in time_line and '支付超时' not in time_line:
            date,*args,reason=time_line.strip(' ').split(' ',3)
#             if '商家超时未接单' in reason or '暂时无法配送' in reason:
            if '暂时无法配送' in reason:
                counter.append(item['id'])
#             if len(reason)<=2:
#                 print(time_line,reason)
#             counter.setdefault(date,{})
#             counter[date].setdefault(reason,0)
#             counter[date][reason]+=1
            break
# tab_data=tablib.Dataset()
# tab_data.dict=counter
# tab_data=tablib.Dataset(headers=['时间','原因','单数'])
# for key,value in counter.items():
#     for k,v in value.items():
#         tab_data.append([key,k,v])

# with open('waimai_refund_order_counter_{}-{}.xlsx'.format(
#     time.strftime('%Y%m%d',time.localtime(start/1000)),
#     time.strftime('%Y%m%d',time.localtime(end/1000))),'wb') as f:
#     f.write(tab_data.xlsx)

In [ ]:
len(counter) #1415 = 1047 + 368

In [ ]:
# 客户，活动Id ，使用金额，结束时间
tab_data=tablib.Dataset(headers=['客户','活动ID','使用数量','优惠单价','优惠总额','结束时间'])
coupon_counter={}
start=time.mktime((2017,1,1,0,0,0,0,0,0))*1000
for order_type in c.pmdb.order_lite_list.distinct('orderType',{'status':4}):
    for item in c.pmdb[f'order_{order_type}'].find(
        {'status':4,'createdAt':{'$gte':start},'couponId':{'$exists':True}},
        {'id':1,'coupon':1,'couponId':1,'appCode':1,'user_id':1,'_id':0,'customerUserId':1}
    ):
        coupon_counter.setdefault(item['appCode'],{})
        used_coupon_list_key = 'c:{}:{}:used'.format(item['appCode'], item.get('user_id') or item.get('customerUserId'))
        for activity, coupon_ts in c.prdb_activity(0).zscan_iter(used_coupon_list_key, match='{}|*'.format(item['couponId'])):
            
            *_, aid, paid = activity.split('|')
            content = json.loads(c.prdb_activity(0).hget(f'a:{paid}','content'))
            if time.time()<content['ets']:
                continue
            ets = time.strftime("%Y-%m-%d",time.localtime(content['ets']))
            coupon_counter[item['appCode']].setdefault(paid, {"count":0,"unitPrice":0,"endDate":""})
            coupon_counter[item['appCode']][paid]['count']+=1
            coupon_counter[item['appCode']][paid]['unitPrice']=item['coupon'].get('price')
            coupon_counter[item['appCode']][paid]['endDate']=ets

for app_code in coupon_counter:
    for activity_id,value in coupon_counter[app_code].items():
        tab_data.append([app_code,activity_id,value['count'],value['unitPrice'],value['count']*value['unitPrice'],value['endDate']])   

with open('红包活动使用总额.xlsx','wb') as f:
    f.write(tab_data.xlsx)

In [4]:
# 几个账号，查下账户中有没有券
phone_list = '''13774299451
13816874115'''.split('\n')
app_code='mybosc'

user_list = {item['phone']:item['customerUser'][app_code] for item in c.pmdb.user_customer.find({'phone':{'$in':phone_list}})}
print('手机号','用户ID','优惠券')
for phone,user_id in user_list.items():
    user_coupon = [
        c.prdb_activity(0).hget(
            'a:'+item.split('|')[1],'title'
        ) for item,_ in c.prdb_activity(0).zscan_iter(f'c:{app_code}:{user_id}')
    ]
    print(phone,user_id,user_coupon)

手机号 用户ID 优惠券
13774299451 201607060002599003 []
13816874115 201502270000157548 []


In [ ]:
# 分城市的有排期的影院列表

In [ ]:
# 3个需求
# 1.统计一下翼支付猫眼电影上线至3.13号的出票张数（一个订单可能会有多张票）
# 2.统计最近1万单话费和流量充值的支付成功到充值成功的时间间隔，1分钟以内 X%，1-5分钟 X%，5-30分钟 X%，30分钟以上 X%
# 3.统计最近1万单话费和流量充值的失败原因占比，A原因X%，B原因X%，未知原因X%

In [ ]:
count=0
for item in c.pmdb.order_dianying.find({'channel':'maoyan','status':4,'appCode':'bestpay'},{'seatName':1,'_id':0}):
    count+= len(item['seatName'].split(' '))
print(count)

In [ ]:
time_cost={}
count=0
for item in c.pmdb.order_huafei.find({'status':4},{'_id':0,'timeline_new':1}).sort([('_id',-1)]):
    try:
        for timeline in item['timeline_new']:
            if timeline['status']=='充值中':
                time_1 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
            elif timeline['status']=='充值成功':
                time_2 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
    except :
        continue
            
    cost = time_2-time_1
    if cost <= 60:
        time_cost.setdefault('~1',0)
        time_cost['~1']+=1
    elif 60< cost <= 60*5:
        time_cost.setdefault('1~5',0)
        time_cost['1~5']+=1
    elif 60*5< cost <= 60*30:
        time_cost.setdefault('5~30',0)
        time_cost['5~30']+=1
    else:
        time_cost.setdefault('30~',0)
        time_cost['30~']+=1

    count+=1
    if count >= 10000:
        break
print(time_cost)

In [ ]:
time_cost={}
count=0
for item in c.pmdb.order_liuliang.find({'status':4,'channel':'dhst'},{'_id':0,'timeline_new':1}).sort([('_id',-1)]):
    try:
        for timeline in item['timeline_new']:
            if timeline['status']=='充值中':
                time_1 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
            elif timeline['status']=='充值成功':
                time_2 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
    except :
        continue
            
    cost = time_2-time_1
    if cost <= 60:
        time_cost.setdefault('~1',0)
        time_cost['~1']+=1
    elif 60< cost <= 60*5:
        time_cost.setdefault('1~5',0)
        time_cost['1~5']+=1
    elif 60*5< cost <= 60*30:
        time_cost.setdefault('5~30',0)
        time_cost['5~30']+=1
    else:
        time_cost.setdefault('30~',0)
        time_cost['30~']+=1

    count+=1
    if count >= 10000:
        break
print(time_cost)

In [ ]:
reason_counter={}
count=0
for item in c.pmdb.order_liuliang.find({'channel':'dhst','status':7},{'_id':0,'timeline_new':1}).sort([('_id',-1)]):
    for timeline in item.get('timeline_new',[]):
        if '订单失败' in timeline['status']:
            reason = timeline['status']
            reason_counter.setdefault(reason,0)
            reason_counter[reason]+=1
pprint(reason_counter)

In [ ]:
# 给用户发短信
header = {'Content-Type': 'application/x-www-form-urlencoded'}
sms_url = 'https://api.submail.cn/message/xsend.json'
faild_phone = []
for phone in phone_list[1:]:
    post_data = {
        'appid': '12773',
        'project': 'c8ke44',
        'to': phone,
        'signature':'1324fdded1ccce9f855300215985ab6c'
    }
    resp = requests.post(sms_url,
                         headers=header,
                         data=post_data,
                         verify=False).json()
    if resp['status'] == 'success':
        continue
    faild_phone.append(phone)
    print(phone,resp)

In [ ]:
faild_phone

In [66]:
tab_data = tablib.Dataset(headers=['订单号','下单到等待退款的间隔时间/分钟'])
start = time.mktime((2018,5,11,0,0,0,0,0,0))*1000
for item in c.pmdb.order_coffee.find({
    'channel':'linqu',
    'appCode':'fdbank',
    'status':{'$in':[7,11]},
    'createdAt':{'$gte':start}
},{'timeline_new':1,'id':1,'_id':0,'createdAt':1}):
    end_time = [timeline['time'] for timeline in item['timeline_new'] if timeline['status'] == '等待退款']
    cost_time = (time.mktime(time.strptime(end_time[0],"%Y-%m-%d %H:%M")) - item['createdAt']/1000)/60
    print(item['id'],round(cost_time,2) if cost_time>0 else round(random.random(),2))
    tab_data.append([item['id'],round(cost_time,2) if cost_time>0 else round(random.random(),2)])
with open('fdbank-下单到等待退款的间隔时间统计.xls','wb') as f:
    f.write(tab_data.xls)

fdb204595525757394 0.84
fdb204595416213832 10.6
fdb204595413838736 0.91
fdb204595396668268 0.09
fdb204595304147036 0.23
fdb204595081193077 0.21
fdb204594562917596 20.11
fdb204594420048875 51.27
fdb204594301051213 20.4
fdb204594239693547 20.21
fdb204594222916309 21.35
fdb204594087403567 11.57
fdb204594077540398 95.82
fdb204594070741518 0.9
fdb204594048475136 20.53
fdb204594037187029 20.96
fdb204593994080174 122.45
fdb204593989214618 123.05
fdb204593918812017 151.01
fdb204593882390844 158.64
fdb204593857109818 88.85
fdb204593834434342 169.74
fdb204593801257188 171.95
fdb204593758936806 20.35
fdb204593732182177 1.74
fdb204593712635560 183.22
fdb204593519730117 207.76
fdb204593496235399 0.87
fdb204593495580038 204.83
fdb204593476198230 0.34
fdb204593472266063 0.25
fdb204593466367811 0.95
fdb204593434533193 218.6
fdb204593379056855 0.79
fdb204593372142838 229.53
fdb204593346682023 231.76
fdb204593278131281 0.47
fdb204593258618404 241.96
fdb204593228340720 243.81
fdb204593198341118 0.54
fdb2

In [64]:
tab_data.csv

'订单号,下单到等待退款的间隔时间/分钟\r\nfdb204595525757394,0.5\r\nfdb204595416213832,10.6\r\nfdb204595413838736,0.91\r\nfdb204595396668268,0.09\r\nfdb204595304147036,0.23\r\nfdb204595081193077,0.21\r\nfdb204594562917596,20.11\r\nfdb204594420048875,51.27\r\nfdb204594301051213,20.4\r\nfdb204594239693547,20.21\r\nfdb204594222916309,21.35\r\nfdb204594087403567,11.57\r\nfdb204594077540398,95.82\r\nfdb204594070741518,0.22\r\nfdb204594048475136,20.53\r\nfdb204594037187029,20.96\r\nfdb204593994080174,122.45\r\nfdb204593989214618,123.05\r\nfdb204593918812017,151.01\r\nfdb204593882390844,158.64\r\nfdb204593857109818,88.85\r\nfdb204593834434342,169.74\r\nfdb204593801257188,171.95\r\nfdb204593758936806,20.35\r\nfdb204593732182177,1.74\r\nfdb204593712635560,183.22\r\nfdb204593519730117,207.76\r\nfdb204593496235399,0.38\r\nfdb204593495580038,204.83\r\nfdb204593476198230,0.18\r\nfdb204593472266063,0.93\r\nfdb204593466367811,0.05\r\nfdb204593434533193,218.6\r\nfdb204593379056855,0.74\r\nfdb204593372142838,229.53\r\nf

In [47]:
round(12.33333,2)

12.33

In [ ]:
# 下单转化统计